# ENB0 - IR

In [5]:
%cd ~/africa_poverty_clean/
%load_ext autoreload
%autoreload 2
%matplotlib inline

/home/jupyter/africa_poverty_clean


In [11]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from glob import glob
from batchers import dataset_constants, tfrecord_paths_utils
from models import processing
from models.loss import r2
from models.checkpoint import CustomModelCheckpoint
import datetime

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [7]:
!pwd

/home/jupyter/africa_poverty_clean


In [28]:
# DHS_TFRECORDS_PATH_ROOT = 'data/dhs_tfrecords/' #local
DHS_TFRECORDS_PATH_ROOT = 'gcs/dhs_tfrecords/' #VM
CSV_PATH = 'data/dhs_clusters.csv' 
CHECKPOINT_PATH = 'models/checkpoints/' 

CLUSTERS_DF = pd.read_csv(CSV_PATH, float_precision='high', index_col=False)
IR_BANDS = ['NIR', 'SWIR1', 'SWIR2']
RGB_BANDS = ['RED', 'GREEN', 'BLUE']
MEANS = dataset_constants._MEANS_DHS
STDS = dataset_constants._STD_DEVS_DHS
BATCH_SIZE = 4
DATASET = 'DHS_OOC_A'
SHUFFLE = 8
PREFETCH = 2
EPOCHS = 50 # CHANGE
STEPS_PER_EPOCH = 2949
VALIDATION_STEPS = 977 #has to be <= (size of val_ds / batch_size)  

### NOTE: (size of dataset / batch size) has to be >= steps_per_epoch * epochs!!!!!
### DHS_OOC_A's train, val, test =  11,797, 3,909, 3,963

In [31]:
def process_datasets(dataset, shuffle, batch_size, prefetch, epochs):
    ''' prepares train_ds, val_ds and test_ds'''
    
    train_tfrecord_paths = tfrecord_paths_utils.dhs_ooc(dataset, split="train")
    val_tfrecord_paths = tfrecord_paths_utils.dhs_ooc(dataset, split="val")
    test_tfrecord_paths = tfrecord_paths_utils.dhs_ooc(dataset, split="test")
    
    # for testing - comment out
#     train_tfrecord_paths = train_tfrecord_paths[:300]
#     val_tfrecord_paths = val_tfrecord_paths[300:400]
#     test_tfrecord_paths = test_tfrecord_paths[0:1]
    print(len(train_tfrecord_paths), len(val_tfrecord_paths), len(test_tfrecord_paths))

    train_ds = tf.data.TFRecordDataset(train_tfrecord_paths, compression_type="GZIP")
    val_ds = tf.data.TFRecordDataset(val_tfrecord_paths, compression_type="GZIP")
    test_ds = tf.data.TFRecordDataset(test_tfrecord_paths, compression_type="GZIP")

    # normalize and resize
    train_ds = train_ds.map(processing.process_tfrecords)
    val_ds = val_ds.map(processing.process_tfrecords)  
    test_ds = test_ds.map(processing.process_tfrecords)

    # train_ds = train_ds.map(processing.augment)
    # val_ds = val_ds.map(processing.augment)  
    # test_ds = test_ds.map(processing.augment)

    train_ds = train_ds.cache()
    train_ds = train_ds.shuffle(shuffle)
    train_ds = train_ds.batch(batch_size)
    train_ds = train_ds.repeat(epochs) # repeats the dataset for the number of epochs 
    train_ds = train_ds.prefetch(prefetch)

    val_ds = val_ds.cache()
    val_ds = val_ds.shuffle(shuffle)
    val_ds = val_ds.batch(batch_size)
    val_ds = val_ds.prefetch(prefetch)

    test_ds = test_ds.cache()
    test_ds = test_ds.shuffle(shuffle)
    test_ds = test_ds.batch(batch_size)
    test_ds = test_ds.prefetch(prefetch)

    return train_ds, val_ds, test_ds

In [26]:
def train_val(model, train_ds, val_ds, checkpoint_path, epochs, steps_per_epoch, validation_steps):
    
    
    now = datetime.datetime.now()
    date_time = now.strftime("%Y%m%d-%H%M%S")
    checkpoint_path = os.path.join(checkpoint_path, f"ENB0_{date_time}")
    
    # creates a folder inside models/checkpoints for checkpoints and csv to be saved
    # folder name: date_time 
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        
    # saves checkpoint at the end of every epoch if val_loss has improved vs the previous epoch 
    print(f"checkpoint_path: {checkpoint_path}")
    cp_callback = CustomModelCheckpoint(
        filepath=checkpoint_path,
        save_best_only=True,
        verbose=2
    )
    
    # logs the outcome of every epoch in a csv file
    csvpath = os.path.join(checkpoint_path, 'model_history_log.csv')
    print(f"csvpath: {csvpath}")
    with open(csvpath, 'a', encoding='utf-8') as f:
        csv_logger = tf.keras.callbacks.CSVLogger(
            csvpath,
            separator=',',
            append=True
        )
    
    hist = model.fit(
        train_ds, epochs=epochs, validation_data=val_ds,
        callbacks=[cp_callback, csv_logger], verbose=2, 
        steps_per_epoch=steps_per_epoch, validation_steps=validation_steps
    )
    return trained_model, hist.history, hist.params

In [39]:
def enb0_instantiate(last_best_weight = None):
    ''' initiates mobilenet model and prints model summary '''

    input_tensor = tf.keras.Input(shape=(224, 224, 3), name = 'images')
    enb0_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor = input_tensor)

    # add new classifier layers
    x = enb0_model.output
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    predictions = tf.keras.layers.Dense(1, activation='linear')(x)

    # create new model with new classifier
    model = tf.keras.Model(inputs=enb0_model.input, outputs=predictions)
    
    adam = tf.keras.optimizers.Adam(
        learning_rate=0.0001,
    )
    
    for layer in model.layers:
        layer.trainable = False

    model.compile(optimizer=adam, loss='mse', metrics=['mse', 'mae', r2])
    print(model.summary())
    
    return model

In [32]:
train_ds, val_ds, test_ds = process_datasets(DATASET, SHUFFLE, BATCH_SIZE, PREFETCH, EPOCHS)

11797 3909 3963


In [33]:
train_ds

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [40]:
model = enb0_instantiate()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 images (InputLayer)            [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_10 (Rescaling)       (None, 224, 224, 3)  0           ['images[0][0]']                 
                                                                                                  
 normalization_5 (Normalization  (None, 224, 224, 3)  7          ['rescaling_10[0][0]']           
 )                                                                                                
                                                                                            

In [42]:
trained_model, history, params = train_val(model, train_ds, val_ds, CHECKPOINT_PATH, EPOCHS, STEPS_PER_EPOCH, VALIDATION_STEPS)

checkpoint_path: models/checkpoints/ENB0_20230307-042802
csvpath: models/checkpoints/ENB0_20230307-042802/model_history_log.csv
Epoch 1/50


KeyboardInterrupt: 